In [1]:
# -*- coding:utf-8 -*-
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import *
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
column_list = "label,channel_id,coin,timestamp,length,coin_seq,feature_seq,pre_1h_return,pre_1h_price,pre_1h_price_avg,pre_1h_volume,pre_1h_volume_avg,pre_1h_volume_sum,pre_1h_volume_tb,pre_1h_volume_quote,pre_1h_volume_quote_tb,pre_3h_return,pre_3h_price,pre_3h_price_avg,pre_3h_volume,pre_3h_volume_avg,pre_3h_volume_sum,pre_3h_volume_tb,pre_3h_volume_tb_avg,pre_3h_volume_tb_sum,pre_3h_volume_quote,pre_3h_volume_quote_avg,pre_3h_volume_quote_sum,pre_3h_volume_quote_tb,pre_3h_volume_quote_tb_avg,pre_3h_volume_quote_tb_sum,pre_6h_return,pre_6h_price,pre_6h_price_avg,pre_6h_volume,pre_6h_volume_avg,pre_6h_volume_sum,pre_6h_volume_tb,pre_6h_volume_tb_avg,pre_6h_volume_tb_sum,pre_6h_volume_quote,pre_6h_volume_quote_avg,pre_6h_volume_quote_sum,pre_6h_volume_quote_tb,pre_6h_volume_quote_tb_avg,pre_6h_volume_quote_tb_sum,pre_12h_return,pre_12h_price,pre_12h_price_avg,pre_12h_volume,pre_12h_volume_avg,pre_12h_volume_sum,pre_12h_volume_tb,pre_12h_volume_tb_avg,pre_12h_volume_tb_sum,pre_12h_volume_quote,pre_12h_volume_quote_avg,pre_12h_volume_quote_sum,pre_12h_volume_quote_tb,pre_12h_volume_quote_tb_avg,pre_12h_volume_quote_tb_sum,pre_24h_return,pre_24h_price,pre_24h_price_avg,pre_24h_volume,pre_24h_volume_avg,pre_24h_volume_sum,pre_24h_volume_tb,pre_24h_volume_tb_avg,pre_24h_volume_tb_sum,pre_24h_volume_quote,pre_24h_volume_quote_avg,pre_24h_volume_quote_sum,pre_24h_volume_quote_tb,pre_24h_volume_quote_tb_avg,pre_24h_volume_quote_tb_sum,pre_36h_return,pre_36h_price,pre_36h_price_avg,pre_36h_volume,pre_36h_volume_avg,pre_36h_volume_sum,pre_36h_volume_tb,pre_36h_volume_tb_avg,pre_36h_volume_tb_sum,pre_36h_volume_quote,pre_36h_volume_quote_avg,pre_36h_volume_quote_sum,pre_36h_volume_quote_tb,pre_36h_volume_quote_tb_avg,pre_36h_volume_quote_tb_sum,pre_48h_return,pre_48h_price,pre_48h_price_avg,pre_48h_volume,pre_48h_volume_avg,pre_48h_volume_sum,pre_48h_volume_tb,pre_48h_volume_tb_avg,pre_48h_volume_tb_sum,pre_48h_volume_quote,pre_48h_volume_quote_avg,pre_48h_volume_quote_sum,pre_48h_volume_quote_tb,pre_48h_volume_quote_tb_avg,pre_48h_volume_quote_tb_sum,pre_60h_return,pre_60h_price,pre_60h_price_avg,pre_60h_volume,pre_60h_volume_avg,pre_60h_volume_sum,pre_60h_volume_tb,pre_60h_volume_tb_avg,pre_60h_volume_tb_sum,pre_60h_volume_quote,pre_60h_volume_quote_avg,pre_60h_volume_quote_sum,pre_60h_volume_quote_tb,pre_60h_volume_quote_tb_avg,pre_60h_volume_quote_tb_sum,pre_72h_return,pre_72h_price,pre_72h_price_avg,pre_72h_volume,pre_72h_volume_avg,pre_72h_volume_sum,pre_72h_volume_tb,pre_72h_volume_tb_avg,pre_72h_volume_tb_sum,pre_72h_volume_quote,pre_72h_volume_quote_avg,pre_72h_volume_quote_sum,pre_72h_volume_quote_tb,pre_72h_volume_quote_tb_avg,pre_72h_volume_quote_tb_sum,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index".split(",")

In [16]:
sample_df = pd.read_csv("test_sample.csv", names=column_list)

In [17]:
sample_df.head()

,label,channel_id,coin,timestamp,length,coin_seq,feature_seq,pre_1h_return,pre_1h_price,pre_1h_price_avg,...,pre_72h_volume_quote_tb_sum,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index
0,0,1219293084,UNI,2021-08-22 17:00:24+00:00,30,NXS\tDREP\tMTH\tWABI\tFIO\tMTH\tOAX\tNXS\tDLT\...,1.48451.3981.51.43049999999999991.3291.43...,-0.003038,57.300152,57.300152,...,8.394792,33.698495,18.239242,2.663055e+06,59.189270,29.224565,13.767188,17.319786,15.629359,11.057383
1,0,1394568162,DNT,2021-11-14 15:00:18+00:00,25,MTH\tEZ\tBRD\tEVX\tWNXM\tSKY\tEZ\tFXS\tVIB\tBR...,0.0590.0590.1330.09150.12250.058989991597...,-0.003661,0.287555,0.287555,...,4.568455,26.992197,11.008106,1.783766e+04,0.274677,24.747581,8.761083,16.140932,13.333309,19.067251
2,0,1480242121,ROSE,2021-06-23 20:18:12+00:00,0,0,0,-0.009558,0.161779,0.161779,...,5.138416,26.541552,11.420027,6.526563e+03,0.183326,21.480570,6.362703,17.319786,10.661867,17.730013
3,0,1219293084,GTO,2021-05-30 17:00:14+00:00,23,NXS\tDLT\tWPR\tVIA\tNEBL\tMDA\tVIB\tSKY\tPNT\t...,2.3232.4833.63100000000000023.0555000000000...,0.018560,0.102340,0.102340,...,6.290786,24.493298,9.237856,3.575287e+03,0.091204,23.596051,8.337902,11.018965,15.273629,18.023587
4,0,1296134866,MTL,2021-08-22 17:00:10+00:00,18,EZ\tDREP\tPOA\tNXS\tIDEX\tVIA\tPIVX\tPPT\tNXS\...,6.77000000000000056.83000000000000126.516.6...,-0.003175,5.940578,5.940578,...,5.562527,27.387110,11.927101,2.701122e+05,5.997579,25.736931,10.278228,15.811779,12.047840,17.660658


In [3]:
input_file = "test_sample.csv" # pos_sample_fg.csv

feat_config = {
    "n_channel": 200,
    "d_channel": 16,
    "n_coin": 1000,
    "d_coin": 16,
    "n_feat": 142,
    "max_length": 50,
    "batch_size": 20,
    "epoch": 3
}

In [18]:
dataset = tf.data.TextLineDataset(input_file)

In [19]:
def parse_split(line):

    parse_res = tf.string_split([line], delimiter=",")
    values = parse_res.values
    label = values[0]
    channel = values[1]
    coin = values[2]
    length = values[4]
    coin_seq = values[5]
    feature_seq = values[6]
    feature_target = values[7:]

    return label, channel, coin, length, coin_seq, feature_seq, feature_target

dataset = dataset.map(parse_split, num_parallel_calls=2)

In [20]:
dataset = dataset.shuffle(1).repeat(feat_config["epoch"]).batch(feat_config["batch_size"])

In [21]:
iterator = dataset.make_one_shot_iterator()

In [22]:
label, channel, coin, length, coin_seq, feature_seq, feature_target = iterator.get_next()

In [26]:
sess = tf.Session()
sess.run(feature_target).shape

(20, 138)